# Пример построения интерактивного графика с использованием библиотеки plotly.express и jupyter_dash

Материалы курсов "Исследования скважин и пластов" и "Гидродинамические исследования скважин" в РГУ нефти и газа имени И.М.Губкина.

Хабибуллин Ринат 2021 г.

версия 0.2 от 25.09.2021

Для построения интерактивных графиков и простых веб приложений может оказаться удобным использовать библиотеку `jupyter-dash`, которая позволяет делать все прямо в тетрадке jupyter или в колабе

Для установки библиотеки надо вызвать в терминале команду
`pip install jupyter-dash`

В колабе это можно сделать выполнив команду 
`!pip install jupyter-dash` в ячейке тетрадки

In [1]:
# в google colab это надо выполнить, чтобы дальше все хорошо сработало
!pip install jupyter-dash

# без колаба не обязательно, если библиотека у вас уже установлена

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import pandas as pd
import numpy as np
import scipy.special as sc

In [3]:
# Решение линейного стока уравнения фильтрации
def pd_ei(rd, td):
  return -1/2*sc.expi(-rd**2 / 4 / td)

In [4]:
df = pd.DataFrame(columns = ['pd', 'td', 'rd'])
df['rd'] = np.logspace(0.1, 4, 100)


app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo test 2"),
    dcc.Graph(id='graph'),
    html.Label([
        "td",
        dcc.Slider(
            id='my-slider',
            min=1e2,
            max=1e8,
            step=1e2,
            value=1e4,
            )
    ]),
    
    dcc.Checklist(
        id='my-check',
        options=[
            {'label': 'semi log', 'value': 'semilog'}
        ],
            value=['semi log', 'semilog']
        )
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input("my-slider", "value"),
    Input("my-check", "value"),
    
)
def update_figure(td, log):
    df['td'] = td
    df['pd'] = pd_ei(df['rd'], df['td'])
    return px.line(
        df, x="rd", y="pd", color = "td",
        render_mode="webgl", title="time = "+str(td),
        log_x = log==['semi log', 'semilog']
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')